In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
df = pd.read_csv("cust_satisfaction.csv")
df.head()

,Gender,Customer Type,Type of Travel,Class,satisfaction,Age,Flight Distance,Inflight entertainment,Baggage handling,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes
0,Male,Loyal Customer,Personal Travel,Eco Plus,neutral or dissatisfied,13,460,5,4,5,25,18.0
1,Male,disloyal Customer,Business travel,Business,neutral or dissatisfied,25,235,1,3,1,1,6.0
2,Female,Loyal Customer,Business travel,Business,satisfied,26,1142,5,4,5,0,0.0
3,Female,Loyal Customer,Business travel,Business,neutral or dissatisfied,25,562,2,3,2,11,9.0
4,Male,Loyal Customer,Business travel,Business,satisfied,61,214,3,4,3,0,0.0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103904 entries, 0 to 103903
Data columns (total 12 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Gender                      103904 non-null  object 
 1   Customer Type               103904 non-null  object 
 2   Type of Travel              103904 non-null  object 
 3   Class                       103904 non-null  object 
 4   satisfaction                103904 non-null  object 
 5   Age                         103904 non-null  int64  
 6   Flight Distance             103904 non-null  int64  
 7   Inflight entertainment      103904 non-null  int64  
 8   Baggage handling            103904 non-null  int64  
 9   Cleanliness                 103904 non-null  int64  
 10  Departure Delay in Minutes  103904 non-null  int64  
 11  Arrival Delay in Minutes    103594 non-null  float64
dtypes: float64(1), int64(6), object(5)
memory usage: 9.5+ MB


In [6]:
df.duplicated().sum()

np.int64(172)

In [7]:
df.drop_duplicates(inplace=True)

In [8]:
df.duplicated().sum()

np.int64(0)

In [9]:
df.isnull().sum()
df.dropna(inplace=True)

In [10]:
df["Customer Type"].value_counts()

Customer Type
Loyal Customer       84517
disloyal Customer    18905
Name: count, dtype: int64

In [11]:
loyal_customer = df[df["Customer Type"] == "Loyal Customer"]
disloyal_customer = df[df["Customer Type"] == "disloyal Customer"]

In [12]:
loyal_customer = loyal_customer.sample(20000)
loyal_customer.shape

(20000, 12)

In [13]:
balance_df = pd.concat([loyal_customer, disloyal_customer],axis=0)
balance_df.shape

(38905, 12)

In [14]:
cat_col = balance_df.select_dtypes(include=["object"])
cat_col.head()

,Gender,Customer Type,Type of Travel,Class,satisfaction
29689,Female,Loyal Customer,Personal Travel,Eco,neutral or dissatisfied
88125,Male,Loyal Customer,Business travel,Eco,neutral or dissatisfied
12013,Male,Loyal Customer,Business travel,Business,satisfied
9454,Female,Loyal Customer,Business travel,Business,neutral or dissatisfied
83226,Female,Loyal Customer,Business travel,Business,satisfied


In [15]:
num_col = balance_df.select_dtypes(exclude=["object"])
num_col.head()

,Age,Flight Distance,Inflight entertainment,Baggage handling,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes
29689,32,550,3,1,3,4,0.0
88125,52,2355,2,4,2,0,40.0
12013,29,2625,5,5,5,35,38.0
9454,30,368,2,1,2,0,0.0
83226,47,2440,4,5,3,16,20.0


In [16]:
pd.get_dummies(cat_col, drop_first=True).astype(int).head()

,Gender_Male,Customer Type_disloyal Customer,Type of Travel_Personal Travel,Class_Eco,Class_Eco Plus,satisfaction_satisfied
29689,0,0,1,1,0,0
88125,1,0,0,1,0,0
12013,1,0,0,0,0,1
9454,0,0,0,0,0,0
83226,0,0,0,0,0,1


In [17]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
ohe = OneHotEncoder(drop="if_binary")
cat_col_encoded = ohe.fit_transform(cat_col).toarray()
cat_col_encoded

array([[0., 0., 1., ..., 1., 0., 0.],
       [1., 0., 0., ..., 1., 0., 0.],
       [1., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 1., 0., ..., 1., 0., 0.],
       [1., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 1., 0., 0.]], shape=(38905, 7))

In [18]:
column_names = list(ohe.get_feature_names_out())
column_names

['Gender_Male',
 'Customer Type_disloyal Customer',
 'Type of Travel_Personal Travel',
 'Class_Business',
 'Class_Eco',
 'Class_Eco Plus',
 'satisfaction_satisfied']

In [19]:
one_hot = pd.DataFrame(cat_col_encoded, columns=column_names)
one_hot.head()

,Gender_Male,Customer Type_disloyal Customer,Type of Travel_Personal Travel,Class_Business,Class_Eco,Class_Eco Plus,satisfaction_satisfied
0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1.0,0.0,0.0,1.0,0.0,0.0,1.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [20]:
one_hot = one_hot.reset_index(drop=True)
num_col = num_col.reset_index(drop=True)

In [21]:
final_df = pd.concat([num_col, one_hot], axis=1)
final_df

,Age,Flight Distance,Inflight entertainment,Baggage handling,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,Gender_Male,Customer Type_disloyal Customer,Type of Travel_Personal Travel,Class_Business,Class_Eco,Class_Eco Plus,satisfaction_satisfied
0,32,550,3,1,3,4,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,52,2355,2,4,2,0,40.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,29,2625,5,5,5,35,38.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
3,30,368,2,1,2,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,47,2440,4,5,3,16,20.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38900,37,596,3,3,3,110,121.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
38901,24,1055,1,5,1,13,10.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
38902,23,192,2,4,2,3,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
38903,30,1995,4,4,4,7,14.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0


In [22]:
final_df.columns

Index(['Age', 'Flight Distance', 'Inflight entertainment', 'Baggage handling',
       'Cleanliness', 'Departure Delay in Minutes', 'Arrival Delay in Minutes',
       'Gender_Male', 'Customer Type_disloyal Customer',
       'Type of Travel_Personal Travel', 'Class_Business', 'Class_Eco',
       'Class_Eco Plus', 'satisfaction_satisfied'],
      dtype='object')

In [23]:
# x and y ---> train test split ---> algo training
from sklearn.model_selection import train_test_split
x = final_df.drop("Customer Type_disloyal Customer", axis=1)
y = final_df["Customer Type_disloyal Customer"]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [24]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(x_train, y_train)

c:\Users\krish\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:470: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [39]:
from sklearn.metrics import accuracy_score
print("accuracy_score", accuracy_score(y_test, model.predict(x_test)))

accuracy_score 0.8412800411258193


In [40]:
from sklearn.metrics import confusion_matrix, classification_report

print(confusion_matrix(y_test, y_pred=model.predict(x_test)))
print(classification_report(y_test, model.predict(x_test)))

[[3496  506]
 [ 729 3050]]
              precision    recall  f1-score   support

         0.0       0.83      0.87      0.85      4002
         1.0       0.86      0.81      0.83      3779

    accuracy                           0.84      7781
   macro avg       0.84      0.84      0.84      7781
weighted avg       0.84      0.84      0.84      7781



In [25]:
# cat_col["Customer Type"].value_counts() 

In [26]:
# dt = {"Loyal Customer":1,"disloyal Customer":0}


In [27]:
# cat_col["Customer Type"] = cat_col["Customer Type"].map(dt)
# cat_col.head()

In [28]:
# cat_col["Gender"].value_counts()

In [29]:
# dt2 = {"Female":1,"Male":0}

In [30]:
# cat_col["Gender"] = cat_col["Gender"].map(dt2)
# cat_col.head()

In [31]:
# cat_col["satisfaction"].value_counts()

In [32]:
# dt3 = {"satisfied":1,"neutral or dissatisfied":0}

In [33]:
# cat_col["satisfaction"] = cat_col["satisfaction"].map(dt3)
# cat_col.head()

In [34]:
# cat_col["Type of Travel"].value_counts()

In [35]:
# dt4 = {"Business travel":1,"Personal Travel":0}
# cat_col["Type of Travel"] = cat_col["Type of Travel"].map(dt4)
# cat_col.head()

In [36]:
# cat_col["Class"].value_counts()

In [37]:
# dt5 = {"Eco":0,"Business":1,"Eco Plus":2}
# cat_col["Class"] = cat_col["Class"].map(dt5)
# cat_col.head()